## Install Prefect and necessary data

In [1]:
! pip install prefect==2.0b5

     |████████████████████████████████| 1.4 MB 24.7 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 5.2 MB/s  eta 0:00:01
     |████████████████████████████████| 872 kB 64.5 MB/s eta 0:00:01
     |████████████████████████████████| 106 kB 71.5 MB/s eta 0:00:01
     |████████████████████████████████| 232 kB 71.1 MB/s eta 0:00:01
     |████████████████████████████████| 54.3 MB 74.5 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 2.8 MB/s  eta 0:00:01
     |████████████████████████████████| 366 kB 73.3 MB/s eta 0:00:01
     |████████████████████████████████| 12.4 MB 63.3 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 4.4 MB/s  eta 0:00:01
     |████████████████████████████████| 80 kB 14.1 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 51.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 67.3 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 69.4 MB/s eta 0:00:01
     |██████████████████████████████

In [2]:
! mkdir data

In [3]:
! wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet -P ./data
! wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet -P ./data

--2022-06-10 09:52:53--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.216.109.59
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.216.109.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11886281 (11M) [binary/octet-stream]
Saving to: ‘./data/fhv_tripdata_2021-01.parquet’

fhv_tripdata_2021-0 100%[===================>]  11.33M  --.-KB/s    in 0.1s    

2022-06-10 09:52:53 (80.7 MB/s) - ‘./data/fhv_tripdata_2021-01.parquet’ saved [11886281/11886281]

--2022-06-10 09:52:53--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 54.231.172.65
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|54.231.172.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10645466 (10M) [binary/octet-stream]
Saving to: ‘./data/fhv_tripdata_2021

In [4]:
! ls -a ./data

.  ..  fhv_tripdata_2021-01.parquet  fhv_tripdata_2021-02.parquet


## Q1. Converting the script to a Prefect flow

#### Check homework.py script

In [5]:
! python homework.py

The mean duration of training is 16.2472533682457
The mean duration of validation is 16.859265811074096
The shape of X_train is (1109826, 525)
The DictVectorizer has 525 features
The MSE of training is: 10.528519403243587
The MSE of validation is: 11.014288317703189


In [19]:
%%writefile homework_prefect_q1.py

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from prefect import flow, task
from prefect import get_run_logger
from prefect.task_runners import SequentialTaskRunner


@task
def read_data(path):
    df = pd.read_parquet(path)
    return df

@task
def prepare_features(df, categorical, train=True):
    logger = get_run_logger()
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    mean_duration = df.duration.mean()
    if train:
        logger.info(f"The mean duration of training is {mean_duration}")
    else:
        logger.info(f"The mean duration of validation is {mean_duration}")
    
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    return df

@task
def train_model(df, categorical):
    logger = get_run_logger()

    train_dicts = df[categorical].to_dict(orient='records')
    dv = DictVectorizer()
    X_train = dv.fit_transform(train_dicts) 
    y_train = df.duration.values

    logger.info(f"The shape of X_train is {X_train.shape}")
    logger.info(f"The DictVectorizer has {len(dv.feature_names_)} features")

    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_train)
    mse = mean_squared_error(y_train, y_pred, squared=False)
    logger.info(f"The MSE of training is: {mse}")
    return lr, dv

@task
def run_model(df, categorical, dv, lr):
    logger = get_run_logger()
    
    val_dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(val_dicts) 
    y_pred = lr.predict(X_val)
    y_val = df.duration.values

    mse = mean_squared_error(y_val, y_pred, squared=False)
    logger.info(f"The MSE of validation is: {mse}")
    return

@flow(task_runner=SequentialTaskRunner())
def main(train_path: str = './data/fhv_tripdata_2021-01.parquet', 
           val_path: str = './data/fhv_tripdata_2021-02.parquet'):

    categorical = ['PUlocationID', 'DOlocationID']

    df_train = read_data(train_path)
    df_train_processed = prepare_features(df_train, categorical)

    df_val = read_data(val_path)
    df_val_processed = prepare_features(df_val, categorical, False)

    # train the model
    lr, dv = train_model(df_train_processed, categorical).result()
    run_model(df_val_processed, categorical, dv, lr)


if __name__ == '__main__':
    main()

Writing homework_prefect_q1.py


In [17]:
! python homework_prefect_q1.py

13:02:13.743 | INFO    | prefect.engine - Created flow run 'nocturnal-horse' for flow 'main'
13:02:13.744 | INFO    | Flow run 'nocturnal-horse' - Using task runner 'SequentialTaskRunner'
13:02:13.759 | WARNING | Flow run 'nocturnal-horse' - No default storage is configured on the server. Results from this flow run will be stored in a temporary directory in its runtime environment.
13:02:13.828 | INFO    | Flow run 'nocturnal-horse' - Created task run 'read_data-4c7f9de4-0' for task 'read_data'
13:02:18.997 | INFO    | Task run 'read_data-4c7f9de4-0' - Finished in state Completed()
13:02:19.040 | INFO    | Flow run 'nocturnal-horse' - Created task run 'prepare_features-4ee39d9f-0' for task 'prepare_features'
13:02:19.258 | INFO    | Task run 'prepare_features-4ee39d9f-0' - The mean duration of training is 16.2472533682457
13:02:26.484 | INFO    | Task run 'prepare_features-4ee39d9f-0' - Finished in state Completed()
13:02:26.524 | INFO    | Flow run 'nocturnal-horse' - Created task run

## Q2. Parameterizing the flow

In [54]:
%%writefile homework_prefect_q2.py

from datetime import datetime, date
import dateutil.relativedelta
import os
import requests

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from prefect import flow, task
from prefect import get_run_logger
from prefect.task_runners import SequentialTaskRunner


DATA_URL = 'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year}-{month}.parquet'
DATA_DIR = './data'
LAST_DATA_DATE = date(2022, 3, 1)


@task
def read_data(path):
    df = pd.read_parquet(path)
    return df


@task
def prepare_features(df, categorical, train=True):
    logger = get_run_logger()
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    mean_duration = df.duration.mean()
    if train:
        logger.info(f"The mean duration of training is {mean_duration}")
    else:
        logger.info(f"The mean duration of validation is {mean_duration}")
    
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    return df


@task
def train_model(df, categorical):
    logger = get_run_logger()

    train_dicts = df[categorical].to_dict(orient='records')
    dv = DictVectorizer()
    X_train = dv.fit_transform(train_dicts) 
    y_train = df.duration.values

    logger.info(f"The shape of X_train is {X_train.shape}")
    logger.info(f"The DictVectorizer has {len(dv.feature_names_)} features")

    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_train)
    mse = mean_squared_error(y_train, y_pred, squared=False)
    logger.info(f"The MSE of training is: {mse}")
    return lr, dv


@task
def run_model(df, categorical, dv, lr):
    logger = get_run_logger()
    
    val_dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(val_dicts) 
    y_pred = lr.predict(X_val)
    y_val = df.duration.values

    mse = mean_squared_error(y_val, y_pred, squared=False)
    logger.info(f"The MSE of validation is: {mse}")
    return


@task
def get_paths(date: (None, str)):
    date = datetime.now() if not date else datetime.strptime(date, '%Y-%m-%d')
    
    train_date = date - dateutil.relativedelta.relativedelta(months=2)
    val_date = date - dateutil.relativedelta.relativedelta(months=1)
    
    train_path = download_source_data(year=train_date.year, month=train_date.month)
    val_path = download_source_data(year=val_date.year, month=val_date.month)
    
    return train_path, val_path
    

def download_source_data(year: int, month: int):
    logger = get_run_logger()
    
    if LAST_DATA_DATE.year < year or (LAST_DATA_DATE.year == year and LAST_DATA_DATE.month < month):
        raise ValueError(f"No taxi data for such date: {year}-{month}")
    
    url = DATA_URL.format(year=year, month=str(month).zfill(2))
    path = os.path.join(DATA_DIR, os.path.basename(url))
    logger.info(f"Downloading file: {url}")
    
    r = requests.get(url)
    with open(path, 'wb') as f:
        f.write(r.content)
    
    return path


@flow(task_runner=SequentialTaskRunner())
def main(date=None):
    train_path, val_path = get_paths(date).result()
    
    categorical = ['PUlocationID', 'DOlocationID']

    df_train = read_data(train_path)
    df_train_processed = prepare_features(df_train, categorical)

    df_val = read_data(val_path)
    df_val_processed = prepare_features(df_val, categorical, False)

    # train the model
    lr, dv = train_model(df_train_processed, categorical).result()
    run_model(df_val_processed, categorical, dv, lr)


if __name__ == '__main__':
    main(date="2021-08-15")

Overwriting homework_prefect_q2.py


In [55]:
! python homework_prefect_q2.py

16:58:49.695 | INFO    | prefect.engine - Created flow run 'wooden-coucal' for flow 'main'
16:58:49.696 | INFO    | Flow run 'wooden-coucal' - Using task runner 'SequentialTaskRunner'
16:58:49.843 | INFO    | Flow run 'wooden-coucal' - Created task run 'get_paths-6e696e34-0' for task 'get_paths'
16:58:49.882 | INFO    | Task run 'get_paths-6e696e34-0' - Downloading file: https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-06.parquet
16:58:50.166 | INFO    | Task run 'get_paths-6e696e34-0' - Downloading file: https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-07.parquet
16:58:50.525 | INFO    | Task run 'get_paths-6e696e34-0' - Finished in state Completed()
16:58:50.557 | INFO    | Flow run 'wooden-coucal' - Created task run 'read_data-4c7f9de4-0' for task 'read_data'
16:58:56.454 | INFO    | Task run 'read_data-4c7f9de4-0' - Finished in state Completed()
16:58:56.494 | INFO    | Flow run 'wooden-coucal' - Created task run 'prepare_features-4ee39d9f-0' for task 'prepa

## Q3. Saving the model and artifacts

In [18]:
%%writefile homework_prefect_q3.py

from datetime import datetime, date
import dateutil.relativedelta
import os
import requests
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import mlflow

from prefect import flow, task
from prefect import get_run_logger
from prefect.task_runners import SequentialTaskRunner


DATA_URL = 'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year}-{month}.parquet'
DATA_DIR = './data'
LAST_DATA_DATE = date(2022, 3, 1)


@task
def read_data(path):
    df = pd.read_parquet(path)
    return df


@task
def prepare_features(df, categorical, train=True):
    logger = get_run_logger()
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    mean_duration = df.duration.mean()
    if train:
        logger.info(f"The mean duration of training is {mean_duration}")
    else:
        logger.info(f"The mean duration of validation is {mean_duration}")
    
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    return df


@task
def train_model(df, categorical, date: str):
    logger = get_run_logger()
    
    train_dicts = df[categorical].to_dict(orient='records')
    dv = DictVectorizer()
    X_train = dv.fit_transform(train_dicts) 
    y_train = df.duration.values

    logger.info(f"The shape of X_train is {X_train.shape}")
    logger.info(f"The DictVectorizer has {len(dv.feature_names_)} features")

    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_train)
    
    mse = mean_squared_error(y_train, y_pred, squared=False)
    logger.info(f"The MSE of training is: {mse}")
    mlflow.log_metric("mse", mse)

    return lr, dv


@task
def run_model(df, categorical, dv, lr):
    logger = get_run_logger()
    
    val_dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(val_dicts) 
    y_pred = lr.predict(X_val)
    y_val = df.duration.values

    mse = mean_squared_error(y_val, y_pred, squared=False)
    logger.info(f"The MSE of validation is: {mse}")
    return


@task
def get_paths(date: (None, str)):
    date = datetime.now() if not date else datetime.strptime(date, '%Y-%m-%d')
    
    train_date = date - dateutil.relativedelta.relativedelta(months=2)
    val_date = date - dateutil.relativedelta.relativedelta(months=1)
    
    train_path = download_source_data(year=train_date.year, month=train_date.month)
    val_path = download_source_data(year=val_date.year, month=val_date.month)
    
    return train_path, val_path
    

def download_source_data(year: int, month: int):
    logger = get_run_logger()
    
    if LAST_DATA_DATE.year < year or (LAST_DATA_DATE.year == year and LAST_DATA_DATE.month < month):
        raise ValueError(f"No taxi data for such date: {year}-{month}")
    
    url = DATA_URL.format(year=year, month=str(month).zfill(2))
    path = os.path.join(DATA_DIR, os.path.basename(url))
    
    if os.path.exists(path):
        logger.info(f"Using existing file: {path}")
        return path
    
    logger.info(f"Downloading file: {url}")
    
    r = requests.get(url)
    with open(path, 'wb') as f:
        f.write(r.content)
    
    return path


@flow(task_runner=SequentialTaskRunner())
def main(date=None):
    mlflow.set_tracking_uri("sqlite:////home/ubuntu/mlops-zoomcamp/mlflow/mlflow.db")
    mlflow.set_experiment("nyc-taxi-experiment-w3")
    
    mlflow.sklearn.autolog()
    
    with mlflow.start_run():
    
        train_path, val_path = get_paths(date).result()
        
        categorical = ['PUlocationID', 'DOlocationID']
    
        df_train = read_data(train_path)
        df_train_processed = prepare_features(df_train, categorical)
    
        df_val = read_data(val_path)
        df_val_processed = prepare_features(df_val, categorical, False)
    
        # train the model
        
        lr, dv = train_model(df_train_processed, categorical, date).result()
        
        with open(f"./models/model-{date}.bin", "wb") as f_out:
            pickle.dump(lr, f_out)
            
        with open(f"./preprocessors/preprocessor-{date}.p", "wb") as f_out:
            pickle.dump(dv, f_out)
            
        mlflow.sklearn.log_model(lr, artifact_path="models")
    
        mlflow.log_artifact(f"preprocessors/preprocessor-{date}.p", artifact_path="preprocessors")
        
        run_model(df_val_processed, categorical, dv, lr)
    

if __name__ == '__main__':
    main(date="2021-08-15")

Overwriting homework_prefect_q3.py


In [19]:
! python homework_prefect_q3.py

17:59:21.233 | INFO    | prefect.engine - Created flow run 'poetic-albatross' for flow 'main'
17:59:21.233 | INFO    | Flow run 'poetic-albatross' - Using task runner 'SequentialTaskRunner'
17:59:22.235 | INFO    | Flow run 'poetic-albatross' - Created task run 'get_paths-6e696e34-0' for task 'get_paths'
17:59:22.276 | INFO    | Task run 'get_paths-6e696e34-0' - Using existing file: ./data/fhv_tripdata_2021-06.parquet
17:59:22.276 | INFO    | Task run 'get_paths-6e696e34-0' - Using existing file: ./data/fhv_tripdata_2021-07.parquet
17:59:22.304 | INFO    | Task run 'get_paths-6e696e34-0' - Finished in state Completed()
17:59:22.337 | INFO    | Flow run 'poetic-albatross' - Created task run 'read_data-4c7f9de4-0' for task 'read_data'
17:59:28.267 | INFO    | Task run 'read_data-4c7f9de4-0' - Finished in state Completed()
17:59:28.308 | INFO    | Flow run 'poetic-albatross' - Created task run 'prepare_features-4ee39d9f-0' for task 'prepare_features'
17:59:28.553 | INFO    | Task run 'pre

In [14]:
! ls -hl ./preprocessors

total 16K
-rw-rw-r-- 1 ubuntu ubuntu 13K Jun 12 17:35 preprocessor-2021-08-15.p


## Q4. Creating a deployment with a CronSchedule

In [20]:
! cp homework_prefect_q3.py hw_flow.py

In [23]:
%%writefile hw_prefect_deployment.py

from datetime import datetime, date
import dateutil.relativedelta
import os
import requests
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import mlflow

from prefect import flow, task
from prefect import get_run_logger
from prefect.task_runners import SequentialTaskRunner


DATA_URL = 'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year}-{month}.parquet'
DATA_DIR = './data'
LAST_DATA_DATE = date(2022, 3, 1)


@task
def read_data(path):
    df = pd.read_parquet(path)
    return df


@task
def prepare_features(df, categorical, train=True):
    logger = get_run_logger()
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    mean_duration = df.duration.mean()
    if train:
        logger.info(f"The mean duration of training is {mean_duration}")
    else:
        logger.info(f"The mean duration of validation is {mean_duration}")
    
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    return df


@task
def train_model(df, categorical, date: str):
    logger = get_run_logger()
    
    train_dicts = df[categorical].to_dict(orient='records')
    dv = DictVectorizer()
    X_train = dv.fit_transform(train_dicts) 
    y_train = df.duration.values

    logger.info(f"The shape of X_train is {X_train.shape}")
    logger.info(f"The DictVectorizer has {len(dv.feature_names_)} features")

    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_train)
    
    mse = mean_squared_error(y_train, y_pred, squared=False)
    logger.info(f"The MSE of training is: {mse}")
    mlflow.log_metric("mse", mse)

    return lr, dv


@task
def run_model(df, categorical, dv, lr):
    logger = get_run_logger()
    
    val_dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(val_dicts) 
    y_pred = lr.predict(X_val)
    y_val = df.duration.values

    mse = mean_squared_error(y_val, y_pred, squared=False)
    logger.info(f"The MSE of validation is: {mse}")
    return


@task
def get_paths(date: (None, str)):
    date = datetime.now() if not date else datetime.strptime(date, '%Y-%m-%d')
    
    train_date = date - dateutil.relativedelta.relativedelta(months=2)
    val_date = date - dateutil.relativedelta.relativedelta(months=1)
    
    train_path = download_source_data(year=train_date.year, month=train_date.month)
    val_path = download_source_data(year=val_date.year, month=val_date.month)
    
    return train_path, val_path
    

def download_source_data(year: int, month: int):
    logger = get_run_logger()
    
    if LAST_DATA_DATE.year < year or (LAST_DATA_DATE.year == year and LAST_DATA_DATE.month < month):
        raise ValueError(f"No taxi data for such date: {year}-{month}")
    
    url = DATA_URL.format(year=year, month=str(month).zfill(2))
    path = os.path.join(DATA_DIR, os.path.basename(url))
    
    if os.path.exists(path):
        logger.info(f"Using existing file: {path}")
        return path
    
    logger.info(f"Downloading file: {url}")
    
    r = requests.get(url)
    with open(path, 'wb') as f:
        f.write(r.content)
    
    return path


@flow(task_runner=SequentialTaskRunner())
def main(date=None):
    mlflow.set_tracking_uri("sqlite:////home/ubuntu/mlops-zoomcamp/mlflow/mlflow.db")
    mlflow.set_experiment("nyc-taxi-experiment-w3")
    
    mlflow.sklearn.autolog()
    
    with mlflow.start_run():
    
        train_path, val_path = get_paths(date).result()
        
        categorical = ['PUlocationID', 'DOlocationID']
    
        df_train = read_data(train_path)
        df_train_processed = prepare_features(df_train, categorical)
    
        df_val = read_data(val_path)
        df_val_processed = prepare_features(df_val, categorical, False)
    
        # train the model
        
        lr, dv = train_model(df_train_processed, categorical, date).result()
        
        with open(f"./models/model-{date}.bin", "wb") as f_out:
            pickle.dump(lr, f_out)
            
        with open(f"./preprocessors/preprocessor-{date}.p", "wb") as f_out:
            pickle.dump(dv, f_out)
            
        mlflow.sklearn.log_model(lr, artifact_path="models")
    
        mlflow.log_artifact(f"preprocessors/preprocessor-{date}.p", artifact_path="preprocessors")
        
        run_model(df_val_processed, categorical, dv, lr)

        
from prefect.orion.schemas.schedules import CronSchedule
from prefect.flow_runners import SubprocessFlowRunner
from prefect.deployments import DeploymentSpec


DeploymentSpec(
    flow=main,
    name="model_training",
    schedule=CronSchedule(
        cron="0 9 15 * *",
        timezone="Europe/Belgrade"
    ),
    tags=["ml"] 
)

Overwriting hw_prefect_deployment.py


In [24]:
! prefect deployment create hw_prefect_deployment.py

Loading deployment specifications from python script at 
'hw_prefect_deployment.py'...
Creating deployment 'model_training' for flow 'main'...
Deploying flow script from 
'/home/ubuntu/mlops-zoomcamp/03-orchestration/hw_prefect_deployment.py' using S3
Storage...
Created deployment 'main/model_training'.
View your new deployment with: 

    prefect deployment inspect 'main/model_training'
Created 1 deployments!


## Q5. Viewing the Deployment

In [25]:
! prefect flow-run ls

                                   Flow Runs                                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━┓
┃                                ID ┃ … ┃ Name           ┃ State ┃ When        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━┩
│ 68f79162-ffac-4f60-a0db-f0153db6… │ … │ fervent-dog    │ SCHE… │ in 3 months │
│ e22aa966-9b9c-406a-8d42-51b12416… │ … │ amusing-corgi  │ SCHE… │ in 2 months │
│ 343a49ba-7e52-4ef0-9f2c-b6560cf1… │ … │ offbeat-seal   │ SCHE… │ in 1 month  │
│ 3d6578b9-e136-49f0-8d55-591328ba… │ … │ devious-crane  │ SCHE… │ in 2 days   │
│ d1415a7f-c1ec-49e0-ab4e-c87ee8bd… │ … │ poetic-albatr… │ COMP… │ 51 minutes… │
│ d10b697d-142d-4f38-b633-4d1532e7… │ … │ almond-mantis  │ COMP… │ 57 minutes… │
│ 9fd4f8e2-13dd-44a8-9b17-829e3d1a… │ … │ ebony-eagle    │ COMP… │ 1 hour ago  │
│ 8f74d408-33a7-4c61-a741-749918d5… │ … │ skilled-marten │ FAIL… │ 1 hour ago  │
│ b5c09129-802e-4ec5-b88f-d1

In [27]:
# answer (option "3" was used)
4

4

## Q6. Creating a work-queue

In [42]:
! prefect work-queue preview dd921f99-100b-4052-956a-88b48742213d --hours 10000

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Scheduled Sta… ┃ Run ID                    ┃ Na… ┃ Deployment ID             ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2022-09-15 07… │ 68f79162-ffac-4f60-a0db-… │ fe… │ 6afdb62e-7900-4a87-ba6f-… │
│ 2022-08-15 07… │ e22aa966-9b9c-406a-8d42-… │ am… │ 6afdb62e-7900-4a87-ba6f-… │
│ 2022-07-15 07… │ 343a49ba-7e52-4ef0-9f2c-… │ of… │ 6afdb62e-7900-4a87-ba6f-… │
│ 2022-06-15 07… │ 3d6578b9-e136-49f0-8d55-… │ de… │ 6afdb62e-7900-4a87-ba6f-… │
└────────────────┴───────────────────────────┴─────┴───────────────────────────┘
                            (**) denotes a late run                             


In [43]:
! prefect work-queue ls

                            Work Queues                            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃                                   ID ┃ Name ┃ Concurrency Limit ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ dd921f99-100b-4052-956a-88b48742213d │ taxi │ None              │
└──────────────────────────────────────┴──────┴───────────────────┘
                    (**) denotes a paused queue                    
